In [1]:
%load_ext autoreload
%autoreload 2


import sys
sys.path.append('../')

In [2]:
from Datasets import RecommenderUserSampler
import pandas as pd
import torch
from torch.utils.data import Subset
from Datasets import EmbeddingsDataset
from grid_search import run_experiments
from utils import map_embeddings_to_ratings, split_recommender_data

ImportError: cannot import name 'run_experiments' from 'grid_search' (/home/gabriel/Projects/REBECA/notebooks/../grid_search.py)

In [3]:
image_features = torch.load("../data/flickr/processed/ip-adapters/SDXL/sdxl_image_embeddings.pt", weights_only=True)
ratings_df = pd.read_csv("../data/flickr/processed/ratings.csv")
expanded_features = map_embeddings_to_ratings(image_features, ratings_df)
device = "cuda"

In [4]:
train_val_df, eval_df, shared_images_df = split_recommender_data(
    ratings_df=ratings_df,
    eval_spu=10,
    seed=42
)

Train set size: 183692
Shared images size: 7489
Evaluation set size: 9516


In [5]:
unique_users = ratings_df["worker_id"].unique()
#user_sampler = RecommenderUserSampler(ratings_df=ratings_df, num_users=len(unique_users), samples_per_user=38)
# Generae test rating ids
val_ratings_sampler = RecommenderUserSampler(ratings_df=train_val_df, num_users=len(unique_users), samples_per_user=10)
val_indices = [i for i in val_ratings_sampler]
# Convert to set for faster lookup
val_indices_set = set(val_indices)
# Create mask for training data
train_mask = ~train_val_df.index.isin(val_indices_set)

val_df = train_val_df[~train_mask]
train_df = train_val_df[train_mask]

print(f"Final train set size: {len(train_df)}")
print(f"Final val set size: {len(val_df)}")

# When creating train_df, keep track of the original indices
train_df_final = train_df.reset_index(drop=False).rename(columns={'index': 'train_df_original_index'})
val_df_final = val_df.reset_index(drop=False).rename(columns={'index': 'train_df_original_index'})


train_val_expanded_features = expanded_features[train_val_df.original_index]

Final train set size: 181680
Final val set size: 2012


In [6]:
full_dataset = EmbeddingsDataset(train_val_df, 
                                 image_embeddings=train_val_expanded_features
                                 )
# Create the dataset using the original indices
train_dataset = Subset(full_dataset, train_df_final['train_df_original_index'].tolist())
val_dataset = Subset(full_dataset, val_df_final["train_df_original_index"].tolist())


In [9]:
train_val_expanded_features.shape

torch.Size([183692, 1280])

In [7]:
param_grid = {
    'timesteps': [6000],
    'layers': [2, 4, 8],
    'heads': [8, 16, 32],
    'dim_feedforward':[1024],
    'num_image_tokens': [1, 4],
    'num_user_tokens': [1, 4],
    'learning_rate': [1e-4],
    #'optimizers': ['adamw', 'sgd'],
    'optimizers': ['adamw'],
    #'schedulers': ['reduce_on_plateau', 'cosine'],
    'schedulers': ['reduce_on_plateau'],
    'batch_size': [128],
    #'noise_schedule': ['linear', "squaredcos_cap_v2"],
    'noise_schedule': ['linear'],
    'samples_per_user': [30, 50, 80, 110, 140],
    'objective':["noise-pred"],
    'use_ue': [True, False],
}

savedir = "../weights/sdxl_experiments"

In [10]:
run_experiments(
    train_df=train_df,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    param_grid=param_grid,
    savedir=savedir
                )

Hyperparameter combinations:   0%|          | 0/360 [00:00<?, ?it/s]c:\Users\Gabriel\anaconda3\envs\recgensys-env\lib\site-packages\torch\nn\modules\transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Running configuration: timesteps=6000, layers=2, heads=8, image_tokens=1, user_tokens=1, learning_rate=0.0001, optimizer=adamw, scheduler=reduce_on_plateau, batch_size=128, noise_schedule=linear, samples_per_user=30, objective=noise-pred, use_ue=True


c:\Users\Gabriel\anaconda3\envs\recgensys-env\lib\site-packages\torch\nn\functional.py:5504: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)
Hyperparameter combinations:   0%|          | 1/360 [01:37<9:44:34, 97.70s/it]

Early stopping with best val loss: 0.11733414884656668!
Running configuration: timesteps=6000, layers=2, heads=8, image_tokens=1, user_tokens=1, learning_rate=0.0001, optimizer=adamw, scheduler=reduce_on_plateau, batch_size=128, noise_schedule=linear, samples_per_user=30, objective=noise-pred, use_ue=False


c:\Users\Gabriel\anaconda3\envs\recgensys-env\lib\site-packages\torch\nn\modules\transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
Hyperparameter combinations:   1%|          | 2/360 [03:04<9:03:07, 91.03s/it]

...

Early stopping with best val loss: 0.10329900775104761!
Running configuration: timesteps=6000, layers=8, heads=32, image_tokens=4, user_tokens=4, learning_rate=0.0001, optimizer=adamw, scheduler=reduce_on_plateau, batch_size=128, noise_schedule=linear, samples_per_user=140, objective=noise-pred, use_ue=True


c:\Users\Gabriel\anaconda3\envs\recgensys-env\lib\site-packages\torch\nn\modules\transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
Hyperparameter combinations: 100%|█████████▉| 359/360 [26:55:36<04:31, 271.84s/it]

Early stopping with best val loss: 0.09518501651473343!
Running configuration: timesteps=6000, layers=8, heads=32, image_tokens=4, user_tokens=4, learning_rate=0.0001, optimizer=adamw, scheduler=reduce_on_plateau, batch_size=128, noise_schedule=linear, samples_per_user=140, objective=noise-pred, use_ue=False


c:\Users\Gabriel\anaconda3\envs\recgensys-env\lib\site-packages\torch\nn\modules\transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
Hyperparameter combinations: 100%|██████████| 360/360 [27:04:40<00:00, 270.78s/it]

Early stopping with best val loss: 0.09450408862903714!
Experimentation complete. Results saved to results.csv at ../weights/sdxl_experiments
